In [12]:
import json
import numpy as np
import pandas as pd
import os

In [13]:
contains_bills = []

for i in range(1, 114):
    path = 'data/' + str(i) + '/bills'  #this path needs to be changed
    if os.path.exists(path):
        contains_bills.append(i)
        
print contains_bills

[109, 112, 113]


In [14]:
def find_subject_in_bills(subject):
    res = []
    for i in range(0, len(tmp)):
        if type(tmp[i]) != dict:
            res.append(0)
        else:
            if subject in tmp[i]['subjects']:
                res.append(1)
            else:
                res.append(0)
    return res

def get_subjects(isAmendment, title):
    if type(title) != str: #
        tmp.append(np.nan)
    else:
        if isAmendment:
            if len(title.split()) > 1:
                bill = title.split()[2]
            else:
                bill = title
        else:
            bill = title
        path = "data/" + str(congress_no) + "/bills/" + ''.join([i for i in bill if not i.isdigit()]) + "/" + bill + "/data.json"
        if os.path.exists(path):
            f = open(path, 'r')
            vote = json.loads(f.read(), 'utf-8')
            obj = {}
            if 'subjects' in vote:
                obj['subjects'] = vote['subjects']
            else:
                obj['subjects'] = np.nan
            if len(vote['committees']):
                obj['committee'] = vote['committees'][0]['committee']
            else:
                obj['committee'] = np.nan
            if 'short_title' in vote:
                obj['billTitle'] = vote['short_title']
            else:
                obj['billTitle'] = np.nan
            if 'sponsor' in vote and vote['sponsor']:
                name = vote['sponsor']['name'].split(', ')
                filtered_name = name[1] + "#" + name[0] + "#" + vote['sponsor']['state']
                obj['sponsor'] = filtered_name
            else:
                obj['sponsor'] = np.nan
            tmp.append(obj)            
        else:
            tmp.append(np.nan)

In [15]:
for congress_no in contains_bills:
    for body in ['house', 'senate']:
        tmp = []
        path = 'csv/' + str(congress_no) + body + '.csv'   #this path needs to be changed
        path2 = 'cleanedcsv/' + str(congress_no) + body + '.csv'  #this path needs to be changed
        df = pd.read_csv(path)
        subjects_dict = {'committee': [], 'billTitle': [], 'sponsor': []}
        for i in range(0, len(df.title)):
            get_subjects(df.isAmendment[i], df.title[i])
            
        count_subjects_dict = {}

        for i in range(0, len(tmp)):
            if type(tmp[i]) == dict:
                subjects_dict['committee'].append(tmp[i]['committee'])
                subjects_dict['billTitle'].append(tmp[i]['billTitle'])
                subjects_dict['sponsor'].append(tmp[i]['sponsor'])
                if type(tmp[i]['subjects']) == list:        
                    for j in range(0, len(tmp[i]['subjects'])):
                        if tmp[i]['subjects'][j] in count_subjects_dict:
                            count_subjects_dict[tmp[i]['subjects'][j]] += 1
                        else:
                            count_subjects_dict[tmp[i]['subjects'][j]] = 1
            else:
                subjects_dict['committee'].append(np.nan)
                subjects_dict['billTitle'].append(np.nan)
                subjects_dict['sponsor'].append(np.nan)

        for key in count_subjects_dict:
            subjects_dict[key] = find_subject_in_bills(key)
        
        subjects_df = pd.DataFrame(data=subjects_dict)

        final = pd.concat([df, subjects_df], axis=1)
        final.to_csv(path2, index=False)

//anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (75,100) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [11]:
for key in subjects_dict:
    if len(subjects_dict[key]) != 697:
        print key

billTitle
committee
